# Part I: From Theory to Code: Constructing a BlockEncoding

If you’ve already delved into the theory of Qubitization and Quantum Signal Processing (GSP/GQSP) in our previous tutorial, you know the drill. If you skipped them because you were eager to see code... no judgment, this is a safe space! Here is all the tl;dr you need to catch up and tag along.

At its core, a block encoding embeds a non-unitary matrix $A$ (which might not even be square) into the top-left corner of a larger unitary matrix $U$. If we can implement $U$, we can "apply" $A$ to a quantum state by projecting onto the correct subspace. Mathematically, a unitary $U$ is an $(\alpha, a, \epsilon)$-block-encoding of $A$ if:$$\| A - \alpha (\bra{0}^a \otimes I) U (\ket{0}^a \otimes I) \| \leq \epsilon$$ 
In Qrisp, we turn this mathematical definition into a functional object that you can add, multiply, and transform just like a standard variable.

That's it. That's all the theory you need to grasp for you to follow this series showcasing how to, among other things, apply polynomial filtering to get a better idea of the ground state! How cool does that sound! If you'd like to delve deeper into the theory, check out the theoretically more rigorous introduction in the theoretical tutorial.

In this tutorial we focus on the "How." You will learn to generate block encodings from various sources:

- Constructors: Use ``.from_operator``, ``.from_array``, or the powerful ``.from_LCU`` (Linear Combination of Unitaries).

- Arithmetic with BlockEncodings: Perform "Quantum Algebra" using standard Pythonic syntax. Need to add two matrices? Just use ``A + B``. Want to scale an operator? Boom. ``0.5 * A``. Multiply two block encodings? ``A @ B``... You get the idea.

- Execution and Quantum Resource Analysis: Move from abstraction to reality. ``.apply()`` automatically synthesizes the gates (no hand weaving circuits, I promise!), handling all ancilla management. For simulation, ``.apply_rus`` invokes a Repeat-Until-Success procedure returning the result only if the all ancillas are measured in $\ket{0}$. To peek under the hood and see exactly how many gates of all kind, you can use the ``.resources`` method.

## Constructing your first BlockEncoding
The BlockEncoding class is designed to meet you wherever you are in your workflow. There are three primary ways to bring an operator into the Qrisp environment: :ref:`.from_array`, :ref:`.from_operator`, and :ref:`.from_LCU`. Let's take a look at how each of them work and how we can use them.

### :ref:`.from_array`
If you are coming from a data science or numerical analysis background, you likely have your data in a matrix. In that case, the :ref:`from_array` is the bridge designed for you. Well as long as the matrix is square and its size $N$ must be a power of two. If this is not the case, fear not, you can always pad the reamaining rows and columns accordingly. The method supports various formats including numpy.ndarray, scipy.sparse.csr_array, and csr_matrix. A quick example would look something like this:




In [8]:
import numpy as np
from qrisp import QuantumFloat
from qrisp.block_encodings import BlockEncoding

# A 4x4 matrix (power of two)
A = np.array([[0, 1, 0, 1],
              [1, 0, 0, 0],
              [0, 0, 1, 0],
              [1, 0, 0, 0]])

BE_A = BlockEncoding.from_array(A)
def operand_prep():
    return QuantumFloat(2)
print(BE_A.resources(operand_prep)())

Simulating 3 qubits.. |                                                      | [  0%]

{'cx': 64, 'x': 9, 'u3': 6, 't': 24, 'cz': 5, 'cy': 2, 'h': 16, 'p': 3, 't_dg': 32, 'gphase': 2}


### ``from_operator``

This method is your go-to for standard quantum operators. It block-encodes the Hermitian part of the operator: $\frac{O + O^\dagger}{2}$. Internally, it performs a Pauli decomposition, breaking the operator down into a sum of Pauli strings which are then block encoded.

In [ ]:
from qrisp.block_encodings import BlockEncoding
from qrisp.operators import X, Y

# Define an operator
H = X(0)*X(1) + 0.2*Y(0)*Y(1)

# Create BlockEncoding
BE_H = BlockEncoding.from_operator(H)
print(BE_H.resources(operand_prep)())

#### ``.from_LCU``
The Linear Combination of Unitaries (LCU) protocol is the "manual transmission" of block encoding. It constructs the unitary via the PREP–SEL–PREP† sequence.Normalization: The scaling factor is $\alpha = \sum_i |\alpha_i|$.Sign Handling: If a coefficient is negative, Qrisp automatically adds a phase flip (Z-gate/gphase) to the corresponding unitary.Padding: If your list of coefficients isn't a power of two, Qrisp pads it with zeros.Mechanism: It uses q_switch internally to perform the SEL (Select) operation, applying $U_i$ conditioned on the ancilla state.

In [ ]:
from qrisp import *
from qrisp.block_encodings import BlockEncoding
import numpy as np

# Define the unitaries
def I(qv): pass

def V(qv):
    qv += 1
    gphase(np.pi, qv[0]) # Negative sign

def V_dg(qv):
    qv -= 1
    gphase(np.pi, qv[0]) # Negative sign

unitaries = [I, V, V_dg]
coeffs = np.array([2.0, 1.0, 1.0])

BE_laplace = BlockEncoding.from_lcu(coeffs, unitaries)

#### Arithmetic and Composition of BlockEncodings

Qrisp allows you to combine BlockEncoding objects using standard Python operators. These operations are not just mathematical abstractions; they dynamically construct new quantum circuits based on the underlying LCU or sequencing logic.

##### addition ``__add__``

Combines two block encodings $A$ and $B$ into $A + B$. It uses an additional ancilla qubit to "decide" which block encoding to apply, scaling the total $\alpha$ to $\alpha_A + \alpha_B$.

| Operation     | Python Syntax | Resulting Operator | New α     | Mechanism                                                                 |
|---------------|---------------|--------------------|-----------|---------------------------------------------------------------------------|
| Addition      | A + B         | A+B                | α_A + α_B | Adds an ancilla qubit to select between U_A and U_B.                      |
| Subtraction   | A - B         | A−B                | α_A + α_B | Same as addition, but with a relative π phase flip.                       |
| Scalar Mult.  | c * A         | c⋅A                | c         |                                                                           |
| Negation      | -A            | −A                 | α_A       | Applies a global phase of π to the unitary.                               |
| Matrix Mult.  | A @ B         | A⋅B                | α_A ⋅ α_B | Sequences the unitaries U_B then U_A.                                     |
| Kronecker     | A.kron(B)     | A⊗B                | α_A ⋅ α_B | Applies unitaries in parallel on separate registers.                      |

In [ ]:
BE_sum = BE1 + BE2

##### subtraction ``__sub__``

Similar to addition, but it applies a phase flip to the second operator to effectively compute $A - B$.

In [ ]:
BE_diff = BE1 - BE2

##### scalar multiplication ``__mul__``

EXPLAIN

In [ ]:
### EXAMPLE

##### matrix multiplication ``__matmul__``

EXPLAIN

In [ ]:
### EXAMPLE

##### kronecker product ``__.kron__``

EXPLAIN

In [ ]:
### EXAMPLE

##### negation ``__neg__``

EXPLAIN

In [ ]:
### EXAMPLE

### Qubitization and Chebyshev Polynomials

#### $n$-step Qubitization

Qubitization turns a block encoding of a Hermitian operator $H$ into a new unitary whose eigenvalues are $e^{\pm i \arccos(H/\alpha)}$. Calling .qubitization() wraps your BE into this framework, usually adding a "signal" qubit to handle the reflection $(\mathbb{I} - 2|0\rangle\langle 0|)$.

#### Chebyshev polynomial using ``.chebyshev``

The .chebyshev(k) method returns a block encoding of the $k$-th Chebyshev polynomial $T_k(A)$. This is the engine behind many spectral algorithms and function approximations.
This is an incredibly powerful method for function approximation. It returns a block encoding of $T_k(A)$, where $T_k$ is the $k$-th Chebyshev polynomial of the first kind.If _rescale=True, it returns $T_k(A)$.If _rescale=False, it returns $T_k(A/\alpha)$.

In [ ]:
# Create a 3rd degree Chebyshev polynomial of the operator
BE_cheb = BE.chebyshev(3)